Import required libraries and list objects from the bucket:

In [1]:
import boto3
import duckdb
import pandas as pd
import numpy as np
import pyarrow
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
s3 = boto3.resource('s3',
  endpoint_url = 'https://7a28e49ec5f4a60c66f216392792ac38.r2.cloudflarestorage.com',
  aws_access_key_id = '1c0813c8e198be6695975d424fdeda05',
  aws_secret_access_key = '5276c800edd5049f6b6f10487e4e769e440db4e928d1f29cb7202b5158b49b74'
)
bucket = s3.Bucket('all-nft-transfers-eth-test-v1')
keys = []

print('Objects:')
for item in bucket.objects.all():
#   print(' - ', item.key)
  keys.append(item.key)




Objects:


Get object by key, convert to numpy dataframe and display as table:

In [9]:
link = "SELECT contract_address, block_height, tx_hash, value FROM read_parquet('https://nft-transfers.sqd-datasets.io/"+keys[0]
link+="');"
print(link)
results = duckdb.sql(link).df()
result_df = results.drop_duplicates()
result_df
#results


SELECT contract_address, block_height, tx_hash, value FROM read_parquet('https://nft-transfers.sqd-datasets.io/0000000000-0012737333/transfers.parquet');


,contract_address,block_height,tx_hash,value
0,0xae7a278aad4dba54d5f0df5f3c9d4616cc5c869f,12686162,0x0fe3d4979ebabea908f137f2774fe9447c436692734d...,0
1,0xae7a278aad4dba54d5f0df5f3c9d4616cc5c869f,12686174,0x8ff8663345e490bbe142f060c21d68a2d37fa30188d7...,0
5,0xae7a278aad4dba54d5f0df5f3c9d4616cc5c869f,12686575,0x1d2cc4cef7e6c81801caf9abf404df6815d2f29a01b9...,50000000000000000
6,0xe9c79b33c3a06f5ae7369599f5a1e2ff886e17f0,12686748,0x5d575f41d0642be3dc8b1b0039b9b4cca74b27551db4...,0
7,0xe9c79b33c3a06f5ae7369599f5a1e2ff886e17f0,12686782,0x432f18a5b427bf50145e214a42f6af16b58aeeb4cd8d...,0
...,...,...,...,...
61093,0x6fc355d4e0ee44b292e50878f49798ff755a5bbc,12737333,0x1e62dddf3047dab7ae35e9fbe85d762f08d571db2ca7...,90000000000000000
61094,0x6fc355d4e0ee44b292e50878f49798ff755a5bbc,12737333,0xa9baaed1a04271759701de98eaf393ad3b9dd486e1a2...,180000000000000000
61096,0x6fc355d4e0ee44b292e50878f49798ff755a5bbc,12737333,0x825b35489a7e6e910226f82441b5f990fb9212dde1be...,90000000000000000
61097,0x6fc355d4e0ee44b292e50878f49798ff755a5bbc,12737333,0x821076ecd3e3eeda9c6302d40bc16a2db6cd53515172...,180000000000000000


Get transfer transaction counts for each contract address:

In [18]:
l = len(keys)
start = l-5

tx_counts = {}
blocks = []
txs = []
counts = []
tx_per_block = {}
gas_per_block = {}

for i in range(start,start+1):
#   print(' - ', item.key)
  #keys.append(item.key)

  if keys[i].endswith('.parquet'):
    link = "SELECT contract_address, block_height, tx_hash, gas_spent, value FROM read_parquet('https://nft-transfers.sqd-datasets.io/"+keys[start]
    link+="');"
    result = duckdb.sql(link).df().drop_duplicates()

    for j in range(result['contract_address'].values.size):
      if result['block_height'].values[j] in gas_per_block:
        gas_per_block[result['block_height'].values[j]] += result['gas_spent'].values[j]
      else:
        gas_per_block[result['block_height'].values[j]] = result['gas_spent'].values[j]

      if result['block_height'].values[j] in tx_per_block:
        tx_per_block[result['block_height'].values[j]] += 1
      else:
        tx_per_block[result['block_height'].values[j]] = 1
      blocks.append(result['block_height'].values[j])
      
      if result['contract_address'].values[j] in tx_counts:
        tx_counts[result['contract_address'].values[j]] += 1
      else:
        tx_counts[result['contract_address'].values[j]] = 1

print(tx_counts)
print(tx_per_block)
print(gas_per_block)
   


  

{'0x379f92687e9eb9993522fc2be014667cb7eb3850': 199, '0x3e77f72c852dfa3e4b271d8a0a6132580a58bee3': 38, '0xd37b244c1574a6a0c0caa60d9f90aaa74365ee5b': 19, '0xac0612b59765efaa52dd09eb13dfc6d84eb5a972': 239, '0x6338f47861f267679caa712583f6f9cc28181088': 51, '0xf7b18abfb989ecf0607d43cdb7e7ff36e5ed0784': 96, '0x3995b3eb28fe088725f050806c0dda4389b22323': 498, '0x7fefc8ede996e2539cc3e10c6bc82a675f1d25a9': 15, '0xf72b9492f4a83abd91618c5979d4f0c07fcb4a0c': 12, '0x211c2beeb5be3b8fa44cefb96cff3f92ad58b941': 6, '0x0519f4d802b9eac027d25788c583f7867751393d': 1, '0x0a2db2858ff0972346b42a327ac994b305f9b38b': 56, '0xa9503d0e2d37244816b5b5ffb9d98d4eb129fd3c': 105, '0xca2576fe376f696d7af21f732c3962cd96c2db11': 12, '0x1d02bbfd85015daa34f974943c68384e494791c8': 1, '0xd3e8b59986b4dcf86b9f144b4f26a49c615ec100': 1, '0x55108c3186591b7e8cb7701d6b447c127bdba1d4': 1, '0xb3f2ee873b170b949b0fec9b533f5c54e42fe96b': 1, '0xcdd207bec65c435d6a924f84a8c5c4b9766696fe': 5, '0xdd7cdaa42ff739952be894ec51b8c7cb847ace40': 6, '0x

Create a plot of transfer count per block:

In [16]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=list(tx_per_block.keys()),
    y=list(tx_per_block.values()),
    name = "erc20",
    mode='markers',
    marker_color='red'
   
))



fig.show()